In [1]:
import sys
import pandas as pd

In [2]:
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

    
from src.utils.paths import DATASET_PATH
from src.data.processing import preprocess_data, add_daily_features
from src.features.fourier_transform import fft_features
from src.dataset.windowing import build_train_df

In [3]:
data = pd.read_csv(DATASET_PATH)

In [4]:
data.head()

,일자,충전방식,0시,1시,2시,3시,4시,5시,6시,7시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,2020-01-01,급속,2800,3200,2320,1600,1520,2240,3120,3800,...,12040,11080,12320,11800,9800,8160,6880,6080,4360,3800
1,2020-01-01,완속,700,805,791,259,84,91,168,189,...,595,784,952,1085,1372,903,1218,1197,1148,1078
2,2020-01-02,급속,1400,1240,1000,680,880,1280,2080,4000,...,10720,11960,11280,11080,9840,10040,7800,5600,4800,3840
3,2020-01-02,완속,413,413,133,133,28,70,105,147,...,483,945,1064,1099,1554,1449,1253,1358,1386,1680
4,2020-01-03,급속,2240,1480,1240,640,920,1320,2320,4240,...,11120,11600,11720,11080,9560,9160,7640,6080,5520,4880


1. 데이터 기본 정제 및 타입변환
2. 칼럼 결측치 처리
3. 시간대별 특성 추가(08시 ~ 20시 비율)

In [5]:
df = add_daily_features(preprocess_data(data))

In [6]:
df.head()

,일자,충전방식,0시,1시,2시,3시,4시,5시,6시,7시,...,16시,17시,18시,19시,20시,21시,22시,23시,daily_total,peak_ratio
0,2020-01-01,급속,2800,3200,2320,1600,1520,2240,3120,3800,...,12320,11800,9800,8160,6880,6080,4360,3800,169120,0.753311
1,2020-01-01,완속,700,805,791,259,84,91,168,189,...,952,1085,1372,903,1218,1197,1148,1078,16387,0.528407
2,2020-01-02,급속,1400,1240,1000,680,880,1280,2080,4000,...,11280,11080,9840,10040,7800,5600,4800,3840,155960,0.778148
3,2020-01-02,완속,413,413,133,133,28,70,105,147,...,1064,1099,1554,1449,1253,1358,1386,1680,16240,0.561638
4,2020-01-03,급속,2240,1480,1240,640,920,1320,2320,4240,...,11720,11080,9560,9160,7640,6080,5520,4880,160400,0.759850


In [7]:
train_df = build_train_df(df, lookback=28, horizon=7, top_k=3)

Windows generated for 급속: 1701
Windows generated for 완속: 1701


In [8]:
train_df.head()

,window_end_date,charging_type,y_next7_total,month,dayofweek,mean_28d,std_28d,last_day_usage,peak_ratio_mean,fft_amp_0,fft_freq_0,fft_amp_1,fft_freq_1,fft_amp_2,fft_freq_2,total_power,spectral_entropy,low_freq_power,high_freq_power
0,2020-01-28,급속,990920,1,1,155388.571429,14572.043124,150280,0.763013,186271.465136,0.142857,130537.591754,0.035714,110619.243164,0.107143,8.323995e+10,1.782513,6.993672e+10,1.330322e+10
1,2020-01-28,완속,117663,1,1,16744.750000,2044.146689,16856,0.570670,26775.368308,0.142857,17723.322530,0.035714,12523.750723,0.107143,1.640356e+09,1.800538,1.343571e+09,2.967851e+08
2,2020-01-29,급속,1011120,1,2,154417.142857,14530.843761,141920,0.763327,166505.216155,0.142857,157512.250348,0.035714,99766.614180,0.107143,8.310286e+10,1.833672,6.898599e+10,1.411686e+10
3,2020-01-29,완속,119497,1,2,16771.750000,2044.235844,17143,0.571938,27525.878572,0.142857,17028.333516,0.035714,12775.817843,0.107143,1.642430e+09,1.773266,1.353154e+09,2.892762e+08
4,2020-01-30,급속,1038720,1,3,153880.000000,14740.354716,140920,0.762532,172473.420486,0.035714,151525.945685,0.142857,87855.156519,0.107143,8.523132e+10,1.889306,7.034628e+10,1.488504e+10
